In [147]:
def get_words(filename):
    with open(filename) as f:
        lines = [line.strip() for line in f]
        assert(len(lines) == 1)
        line = lines[0]
        words = line.split(',')
        for i in range(len(words)):
            words[i] = words[i].strip()[1:-1]
        return words

In [148]:
final_words_loc = "/home/harsh/now/wordle/words1.txt"
attempt_words_loc = "/home/harsh/now/wordle/words2.txt"

In [149]:
final_words = get_words(final_words_loc)  # final answer possibilities
# attempt_words = get_words(attempt_words_loc)  # attempted answer possibilities

In [150]:
def get_freq(words):
    freq = {}
    for char in range(97, 123):
        freq[chr(char)] = [0,0,0,0,0] # for 1 occurence, 2 occurences, etc.
    assert(len(freq) == 26)
    for word in words:
        assert(len(word) == 5)
        chs = []
        for ch in word:
            num_ch_yet = chs.count(ch)
            freq[ch][num_ch_yet] += 1
            chs.append(ch)
    return freq

In [151]:
freq = get_freq(final_words)

In [152]:
'''
If I get a yellow char, I am going to include it in every further guess. But in certain cases, trying a word without that yellow char might be more useful in reducing number of possibilities. So look into that later.

words with >1 same char will still be problematic. Deal with them

I am using only final_words to make guesses right now. Using attempt_words might give me even better results. Attempt words can be really useful to reduce number of possibilities.


'''

'\nIf I get a yellow char, I am going to include it in every further guess. But in certain cases, trying a word without that yellow char might be more useful in reducing number of possibilities. So look into that later.\n\nwords with >1 same char will still be problematic. Deal with them\n\nI am using only final_words to make guesses right now. Using attempt_words might give me even better results. Attempt words can be really useful to reduce number of possibilities.\n\n\n'

In [153]:
class Solver:
    def __init__(self, freq, attempt_words):
        self.freq = freq
        self.attempt_words = self.initial_attempt_words(attempt_words=attempt_words)
        self.constraints = {chr(ch):[] for ch in range(97, 123)}
        # for ch in range(97, 123):
        #     self.constraints[ch] = [None] * 5
    
    def solve(self, final_word):
        assert(len(final_word) == 5)
        num_greens = 0 # 5 greens = win
        num_guesses = 0 # max 6 guesses
        while num_greens < 5 and num_guesses < 6:
            # print(f"len of self.attempt_words: {len(self.attempt_words)}")
            guess = self.attempt_words[0]
            num_guesses += 1
            res = self.result(guess, final_word)
            for ch_i in range(len(res)):
                ch, color = res[ch_i]
                self.constraints[ch].append((color, ch_i))
                if color == "green":
                    num_greens += 1
            self.update_attempts_words()
        return num_guesses
    
    
    def running_solve(self):
        num_greens = 0
        num_guesses = 0
        while num_greens < 5 and num_guesses < 6:
            guess = self.attempt_words[0]
            num_guesses += 1
            res = self.running_result(guess)
            print(f"res is {res}")
            for ch_i in range(len(res)):
                ch, color = res[ch_i]
                self.constraints[ch].append((color, ch_i))
                if color == "green":
                    num_greens += 1
            self.update_attempts_words()
        return num_guesses
    
    def running_result(self, guess):
        print(f"guess is {guess}")
        res = [ch for ch in guess]
        inp = input("give res:\n").split(',') # e.g. gray, yellow, gray, yellow, green
        for ch_i in range(len(res)):
            ch = res[ch_i]
            res[ch_i] = (ch, inp[ch_i])
        return res
    
    
    def result(self, guess, final_word):
        res = [(ch, None) for ch in guess]
        for ch_i in range(len(guess)):
            ch = guess[ch_i]
            if ch == final_word[ch_i]:
                res[ch_i] = (ch, "green")
            elif ch in final_word:
                res[ch_i] = (ch, "yellow")
            else:
                res[ch_i] = (ch, "gray")
        return res
    
    
    
    
    def initial_attempt_words(self, attempt_words): # TODO: positional stuff
        new_attempt_words = {word:0 for word in attempt_words}
        for attempt_word in attempt_words:
            assert(len(attempt_word) == 5)
            chs = []
            attempt_word_score = 0
            for ch in attempt_word:
                num_ch_yet = chs.count(ch)
                attempt_word_score += self.freq[ch][num_ch_yet]
                chs.append(ch)
            assert(new_attempt_words[attempt_word] == 0)
            new_attempt_words[attempt_word] = attempt_word_score
        new_attempt_words = sorted(new_attempt_words.items(), key=lambda x: x[1], reverse=True)
        new_attempt_words = [word for word, score in new_attempt_words]
        return new_attempt_words
    
    
    def update_attempts_words(self):
        # updates self.attempt_words based on self.constraints
        new_attempt_words = []
        for attempt_word in self.attempt_words:
            assert(len(attempt_word) == 5)
            to_include = True
            
            for ch, ch_cons in self.constraints.items():
                # ch_cons = self.constraints[ch]
                for ch_con in ch_cons:
                    color, color_index = ch_con
                    if color == "gray" and not (ch not in attempt_word):
                        to_include = False
                    elif color == "yellow" and not (ch in attempt_word and attempt_word[color_index] != ch):
                        to_include = False
                    elif color == "green" and not (attempt_word[color_index] == ch):
                        to_include = False
            
            if to_include:
                new_attempt_words.append(attempt_word)
        self.attempt_words = new_attempt_words
        
    
    
    
    
    
    
    


In [154]:
solver = Solver(freq, final_words)

In [155]:
def score_solver(final_words):
    total_guesses = 0 # calc total guesses made yet
    total_misses = 0 # calc total misses yet
    total_hits = 0
    for final_word_i in range(len(final_words)):
        final_word = final_words[final_word_i]
        solver = Solver(freq, final_words)
        num_guesses = solver.solve(final_word)
        if num_guesses <= 5:
            total_guesses += num_guesses
            total_hits += 1
        elif num_guesses == 6:
            total_misses += 1
        # print(f"final_word_index is {final_word_i}")
    return (total_guesses, total_hits, total_misses)

In [159]:
total_guesses, total_hits, total_misses = score_solver(final_words)
print(f"total_guesses: {total_guesses} and total_hits: {total_hits} and total_misses: {total_misses}")

total_guesses: 7614 and total_hits: 2315 and total_misses: 0


In [ ]:
solver = Solver(freq, final_words)
solver.running_solve()

In [158]:
len(final_words)

2315